In [9]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))
from src.dataset_pipeline import load_dataset
import numpy as np

import torch
import numpy as np
from sklearn.metrics import normalized_mutual_info_score, homogeneity_score, silhouette_score
from sklearn.cluster import KMeans

# Assuming these are your custom modules based on the paper's architecture
from src.clust_dr import DR_then_Clust
from src.affinities import NormalizedGaussianAndStudentAffinity, SymmetricEntropicAffinity
from src.clust_dr import Clust_then_DR, DistR

In [10]:
# List of datasets to load
# Excluded 'coil20' due to current server downtime (404/522)
# 'snareseq' requires manual download of multiple files usually, but we can try if implemented.
# We focus on the ones known to work or standard in scanpy/torchvision.
datasets_to_load = ['mnist', 'fmnist', 'pbmc', 'zeisel']

loaded_data = {}

for name in datasets_to_load:
    try:
        print(f"Loading {name}...")
        # Load with default PCA dim 50
        data = load_dataset(name, pca_dim=50)
        loaded_data[name] = data
        
        print(f"Successfully loaded {name}")
        print(f"  X shape: {data['X'].shape}")
        print(f"  Y shape: {data['Y'].shape}")
        print(f"  Original dim: {data['original_dim']}")
        print("-" * 30)
    except Exception as e:
        print(f"Failed to load {name}: {e}")
        print("-" * 30)

Loading mnist...
Loading mnist...
Original shape: (10000, 784)
Applying PCA to 50 dimensions...
Successfully loaded mnist
  X shape: (10000, 50)
  Y shape: (10000,)
  Original dim: 784
------------------------------
Loading fmnist...
Loading fmnist...
Original shape: (10000, 784)
Applying PCA to 50 dimensions...
Successfully loaded fmnist
  X shape: (10000, 50)
  Y shape: (10000,)
  Original dim: 784
------------------------------
Loading pbmc...
Loading pbmc...


/opt/python/lib/python3.13/site-packages/scanpy/preprocessing/_simple.py:176: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["n_genes"] = number


Original shape: (2638, 2000)
Applying PCA to 50 dimensions...
Successfully loaded pbmc
  X shape: (2638, 50)
  Y shape: (2638,)
  Original dim: 2000
------------------------------
Loading zeisel...
Loading zeisel...
Failed to load zeisel: scanpy.datasets.zeisel() not found. Please update scanpy.
------------------------------


In [11]:
# Example access
if 'mnist' in loaded_data:
    X_mnist = loaded_data['mnist']['X']
    Y_mnist = loaded_data['mnist']['Y']
    print("MNIST data ready for DistR.")

MNIST data ready for DistR.


In [ ]:


def get_majority_vote_labels(T, Y):
    """
    Assigns a class label to each prototype based on the majority class 
    of the samples mapped to it (weighted by the transport plan T).
    Operations are fully vectorized on GPU.
    """
    # T: (N_samples, n_prototypes) [GPU]
    # Y: (N_samples,) [CPU/Numpy]
    
    n_prototypes = T.shape[1]
    # Convert Y to GPU tensor once
    Y_tensor = torch.as_tensor(Y, device=T.device, dtype=torch.long)
    n_classes = Y_tensor.max().item() + 1
    
    # One-hot encode Y to aggregate mass: (N_samples, n_classes)
    # We use scatter on GPU
    Y_onehot = torch.zeros((len(Y), n_classes), device=T.device, dtype=T.dtype)
    Y_onehot.scatter_(1, Y_tensor.unsqueeze(1), 1.0)
    
    # Aggregate mass: Project Y onto Prototypes via T^T
    # vote_matrix: (n_prototypes, n_classes)
    # This matrix multiplication is the heavy lifting, kept on GPU
    vote_matrix = torch.mm(T.T, Y_onehot)
    
    # Get majority class for each prototype
    proto_labels = torch.argmax(vote_matrix, dim=1).cpu().numpy()
    return proto_labels

def evaluate_prototypes(Z, T, Y_true):
    """
    Computes metrics defined in the paper [cite: 332-339].
    Transfers to CPU only when necessary for sklearn.
    """
    # Z and T are on GPU
    
    # 1. Homogeneity Score (Evaluated on Samples)
    # Hard assignment: x_i assigned to prototype j with max T_ij
    # Perform argmax on GPU before transfer
    sample_assignments = torch.argmax(T, dim=1).cpu().numpy()
    homogeneity = homogeneity_score(Y_true, sample_assignments)
    
    # Data transfer for K-Means and Silhouette (sklearn is CPU-bound)
    Z_np = Z.detach().cpu().numpy()
    
    # 2. K-Means NMI Score (Evaluated on Prototypes)
    n_classes = len(np.unique(Y_true))
    # Note: If you have cuML installed, KMeans can be run on GPU here.
    kmeans = KMeans(n_clusters=n_classes, random_state=42, n_init=10)
    proto_clusters = kmeans.fit_predict(Z_np)
    
    # Propagate prototype clusters back to samples
    predicted_labels = proto_clusters[sample_assignments]
    nmi = normalized_mutual_info_score(Y_true, predicted_labels)
    
    # 3. Silhouette Score (Weighted by Prototype Mass)
    proto_labels = get_majority_vote_labels(T, Y_true)
    
    # Calculate weights (mass of each prototype) on GPU
    weights = T.sum(dim=0).cpu().numpy()
    
    try:
        # Silhouette requires CPU
        sil = silhouette_score(Z_np, proto_labels, sample_weight=weights)
    except ValueError:
        sil = -1.0
        
    return {
        "Homogeneity": homogeneity,
        "NMI": nmi,
        "Silhouette": sil
    }

def run_experiment(data_dict, dataset_name, n_prototypes=50, device='cuda'):
    """
    Runs DistR and Baselines on a single dataset.
    """
    print(f"=== Running Experiment on {dataset_name} ===")
    print(f"Configuration: N={data_dict['X'].shape[0]}, Prototypes={n_prototypes}, Device={device}")
    
    # OPTIMIZATION: Use float32 for faster GPU computation (Tensor Cores)
    # Create tensor directly on device to avoid CPU->GPU copy
    X = torch.as_tensor(data_dict['X'], dtype=torch.float32, device=device)
    Y = data_dict['Y'] 
    
    # Initialize Affinities
    # Ensure these classes respect the inputs' dtype and device!
    # [cite: 329] Symmetric Entropic Affinity for X
    affinity_X = SymmetricEntropicAffinity(perp=30, verbose=False) 
    # [cite: 329] Student t-distribution for Z
    affinity_Z = NormalizedGaussianAndStudentAffinity(student=True) 
    
    results = {}
    
    # 1. DistR (Ours)
    print(f"\n[1/3] Training DistR (Device: {device})...")
    model_distr = DistR(
        affinity_data=affinity_X,
        affinity_embedding=affinity_Z,
        output_sam=n_prototypes,
        loss_fun="kl_loss",       # [cite: 120]
        optimizer="Adam",         # [cite: 233]
        lr=0.1,
        max_iter=200,             # BCD Iterations [cite: 232]
        device=device,
        init="normal",
        init_T="kmeans"
    )
    Z_distr = model_distr.fit_transform(X)
    metrics_distr = evaluate_prototypes(Z_distr, model_distr.T, Y)
    results['DistR'] = metrics_distr
    print(f"DistR Results: {metrics_distr}")

    # 2. DR -> Clustering
    print(f"\n[2/3] Training DR -> Clustering (Device: {device})...")
    model_drc = DR_then_Clust(
        affinity_data=affinity_X,
        affinity_embedding=affinity_Z,
        output_sam=n_prototypes,
        output_dim=2,
        loss_fun="kl_loss",
        device=device,
        init="normal",
        init_T="kmeans"
    )
    Z_drc = model_drc.fit_transform(X)
    metrics_drc = evaluate_prototypes(Z_drc, model_drc.T, Y)
    results['DR_then_Clust'] = metrics_drc
    print(f"DR->C Results: {metrics_drc}")

    # 3. Clustering -> DR
    print(f"\n[3/3] Training Clustering -> DR (Device: {device})...")
    model_cdr = Clust_then_DR(
        affinity_data=affinity_X,
        affinity_embedding=affinity_Z,
        output_sam=n_prototypes,
        output_dim=2,
        loss_fun="kl_loss",
        device=device,
        init="normal",
        init_T="kmeans"
    )
    Z_cdr = model_cdr.fit_transform(X)
    metrics_cdr = evaluate_prototypes(Z_cdr, model_cdr.T, Y)
    results['Clust_then_DR'] = metrics_cdr
    print(f"C->DR Results: {metrics_cdr}")
    
    return results

if __name__ == "__main__":
    # Auto-detect device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Master Process Device: {device}")
    if device == 'cuda':
        print(f"GPU: {torch.cuda.get_device_name(0)}")
    
    # Placeholder for loaded_data context
    # loaded_data should be a dictionary: {'dataset_name': {'X': np.array, 'Y': np.array}}
    target_dataset = 'mnist'
    
    if 'loaded_data' in locals() and target_dataset in loaded_data:
        data_subset = {
            'X': loaded_data[target_dataset]['X'],
            'Y': loaded_data[target_dataset]['Y']
        }
        
        # Strategy: n_prototypes = n_classes + 20 [cite: 1114]
        n_classes = len(np.unique(data_subset['Y']))
        n_prototypes = n_classes + 20
        
        final_scores = run_experiment(
            data_subset, 
            target_dataset, 
            n_prototypes=n_prototypes, 
            device=device
        )
        
        print("\n=== Final Benchmark Report ===")
        for method, scores in final_scores.items():
            print(f"{method}:")
            for metric, val in scores.items():
                print(f"  {metric}: {val:.4f}")
    else:
        print(f"Warning: Dataset '{target_dataset}' data not found in local context. Script is ready for integration.")

Master Process Device: cuda
GPU: Tesla T4
=== Running Experiment on mnist ===
Configuration: N=10000, Prototypes=30, Device=cuda

[1/3] Training DistR (Device: cuda)...
